# Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

* `population` - the global population.
* `population_growth` - the annual population growth rate, as a percentage.
* `area` - the total land and water area.
* `area_land` - the country's land area in square kilometers.
* `area_water` — the country's waterarea in square kilometers.
* `birth_rate` - the country's birth rate, or the number of births per year per 1,000 people.
* `death_rate` - the country's death rate, or the number of death per year per 1,000 people.

In this guided project, we'll use SQL in Jupyter Notebook to analyze data from this database.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:

%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [4]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


In [5]:
%%sql
SELECT name
  FROM facts
 WHERE population IN (SELECT MIN(population)
                      FROM facts
                      );

Done.


name
Antarctica


In [6]:
%%sql
SELECT name
  FROM facts
 WHERE population IN (SELECT MAX(population)
                      FROM facts
                      );

Done.


name
World


In [7]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts
 WHERE name != 'World';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [8]:
%%sql
SELECT AVG(population), AVG(area)
  FROM facts
 WHERE name != 'World';

Done.


AVG(population),AVG(area)
32242666.56846473,555093.546184739


In [9]:
%%sql
SELECT name, population, area
  FROM facts
 WHERE (population > (SELECT AVG(population) 
                      FROM facts
                      WHERE name != 'World') 
        AND area < (SELECT AVG(area)
                    FROM facts
                    WHERE name != 'World'));

Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


## Which country has the most people? Which country has the highest growth rate?

In [10]:
%%sql
SELECT name, population
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                       WHERE name != 'World');

Done.


name,population
China,1367485388


In [11]:
%%sql
SELECT name, population_growth
  FROM facts
 WHERE population_growth == (SELECT MAX(population_growth)
                              FROM facts
                            WHERE name != 'World');

Done.


name,population_growth
South Sudan,4.02


## Which countries have the highest ratios of water to land? Which countries have more water than land?

In [12]:
%%sql
SELECT name, CAST(area_water AS FLOAT) / area AS water_ratio
                
  FROM facts
 WHERE name != 'World'
  ORDER BY water_ratio DESC
 LIMIT 10;

Done.


name,water_ratio
British Indian Ocean Territory,0.9988970588235294
Virgin Islands,0.818848167539267
Puerto Rico,0.35682691610470596
"Bahamas, The",0.2788184438040346
Guinea-Bissau,0.2215916955017301
Malawi,0.20596873839505755
Netherlands,0.1841465469513516
Uganda,0.18228661040997685
Eritrea,0.141156462585034
Liberia,0.13512736937567904


In [13]:
%%sql
SELECT name, area_land, area_water                
  FROM facts
 WHERE area_land < area_water;


Done.


name,area_land,area_water
British Indian Ocean Territory,60,54340
Virgin Islands,346,1564


## Which countries will add the most people to their populations next year?

In [14]:
%%sql
SELECT name, (population * CAST(population_growth AS FLOAT) / 100 + population) AS population_next_year
  FROM facts
 WHERE name != 'World'
  ORDER BY population_next_year DESC
 LIMIT 2;

                 

Done.


name,population_next_year
China,1373639072.246
India,1266966270.1248


## Which countries have a higher death rate than birth rate?

In [15]:
%%sql
SELECT name, death_rate, birth_rate, (birth_rate - death_rate) AS birth_and_deats_diff
  FROM facts
 WHERE death_rate > birth_rate
  ORDER BY birth_and_deats_diff;

Done.


name,death_rate,birth_rate,birth_and_deats_diff
Bulgaria,14.44,8.92,-5.52
Serbia,13.66,9.08,-4.58
Latvia,14.31,10.0,-4.3100000000000005
Lithuania,14.27,10.1,-4.17
Ukraine,14.46,10.72,-3.74
Hungary,12.73,9.16,-3.5700000000000003
Germany,11.42,8.47,-2.9499999999999993
Slovenia,11.37,8.42,-2.9499999999999993
Romania,11.9,9.14,-2.76
Croatia,12.18,9.45,-2.7300000000000004


## Which countries have the highest population/area ratio?

In [16]:
%%sql
SELECT name, (population / area) AS population_area_ratio
  FROM FACTS
 WHERE name != 'World'
  ORDER BY population_area_ratio DESC
 LIMIT 5;

Done.


name,population_area_ratio
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
